# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 12:16PM,259076,19,9227516,Eywa Pharma Inc.,Released
1,Mar 10 2023 12:16PM,259076,19,9220935-BO,Eywa Pharma Inc.,Released
2,Mar 10 2023 12:16PM,259076,19,9238911,Eywa Pharma Inc.,Released
3,Mar 10 2023 12:16PM,259076,19,9243786,Eywa Pharma Inc.,Released
4,Mar 10 2023 12:16PM,259076,19,9243794,Eywa Pharma Inc.,Released
5,Mar 10 2023 12:00PM,259084,18,CAP-33995892,Capsugel,Released
6,Mar 10 2023 11:50AM,259082,10,HH9108a,Hush Hush,Released
7,Mar 10 2023 11:50AM,259082,10,HH9108b,Hush Hush,Released
8,Mar 10 2023 11:50AM,259082,10,HH9108c,Hush Hush,Released
9,Mar 10 2023 11:50AM,259082,10,HH9108d,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,259076,Released,5
23,259079,Released,10
24,259080,Released,1
25,259082,Released,4
26,259084,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259076,NaN,5.0
259079,NaN,10.0
259080,NaN,1.0
259082,NaN,4.0
259084,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259023,0.0,4.0
259024,0.0,1.0
259033,0.0,5.0
259042,0.0,1.0
259052,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259023,0,4
259024,0,1
259033,0,5
259042,0,1
259052,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259023,0,4
1,259024,0,1
2,259033,0,5
3,259042,0,1
4,259052,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259023,,4
1,259024,,1
2,259033,,5
3,259042,,1
4,259052,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.
5,Mar 10 2023 12:00PM,259084,18,Capsugel
6,Mar 10 2023 11:50AM,259082,10,Hush Hush
10,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC"
11,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc."
21,Mar 10 2023 11:40AM,259072,15,"Alliance Pharma, Inc."
31,Mar 10 2023 11:39AM,259033,15,"Person & Covey, Inc."
36,Mar 10 2023 11:39AM,259024,15,"Virtus Pharmaceuticals, LLC"
37,Mar 10 2023 11:38AM,259023,15,"Alliance Pharma, Inc."
41,Mar 10 2023 11:28AM,259073,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.,,5
1,Mar 10 2023 12:00PM,259084,18,Capsugel,,1
2,Mar 10 2023 11:50AM,259082,10,Hush Hush,,4
3,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC",,1
4,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc.",,10
5,Mar 10 2023 11:40AM,259072,15,"Alliance Pharma, Inc.",,10
6,Mar 10 2023 11:39AM,259033,15,"Person & Covey, Inc.",,5
7,Mar 10 2023 11:39AM,259024,15,"Virtus Pharmaceuticals, LLC",,1
8,Mar 10 2023 11:38AM,259023,15,"Alliance Pharma, Inc.",,4
9,Mar 10 2023 11:28AM,259073,10,Eywa Pharma Inc.,1,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.,5,
1,Mar 10 2023 12:00PM,259084,18,Capsugel,1,
2,Mar 10 2023 11:50AM,259082,10,Hush Hush,4,
3,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC",1,
4,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc.",10,
5,Mar 10 2023 11:40AM,259072,15,"Alliance Pharma, Inc.",10,
6,Mar 10 2023 11:39AM,259033,15,"Person & Covey, Inc.",5,
7,Mar 10 2023 11:39AM,259024,15,"Virtus Pharmaceuticals, LLC",1,
8,Mar 10 2023 11:38AM,259023,15,"Alliance Pharma, Inc.",4,
9,Mar 10 2023 11:28AM,259073,10,Eywa Pharma Inc.,5,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.,5,
1,Mar 10 2023 12:00PM,259084,18,Capsugel,1,
2,Mar 10 2023 11:50AM,259082,10,Hush Hush,4,
3,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC",1,
4,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc.",10,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.,5.0,NaN
1,Mar 10 2023 12:00PM,259084,18,Capsugel,1.0,NaN
2,Mar 10 2023 11:50AM,259082,10,Hush Hush,4.0,NaN
3,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC",1.0,NaN
4,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc.",10.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 12:16PM,259076,19,Eywa Pharma Inc.,5.0,0.0
1,Mar 10 2023 12:00PM,259084,18,Capsugel,1.0,0.0
2,Mar 10 2023 11:50AM,259082,10,Hush Hush,4.0,0.0
3,Mar 10 2023 11:47AM,259080,15,"Mizner Bioscience, LLC",1.0,0.0
4,Mar 10 2023 11:46AM,259079,15,"Person & Covey, Inc.",10.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,777230,10.0,1.0
15,2072448,32.0,6.0
16,259056,0.0,1.0
18,259084,1.0,0.0
19,1554356,36.0,40.0
20,259074,0.0,1.0
21,518123,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,777230,10.0,1.0
1,15,2072448,32.0,6.0
2,16,259056,0.0,1.0
3,18,259084,1.0,0.0
4,19,1554356,36.0,40.0
5,20,259074,0.0,1.0
6,21,518123,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,10.0,1.0
1,15,32.0,6.0
2,16,0.0,1.0
3,18,1.0,0.0
4,19,36.0,40.0
5,20,0.0,1.0
6,21,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,10.0
1,15,Released,32.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,36.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Executing,1.0,6.0,1.0,0.0,40.0,1.0,2.0
Released,10.0,32.0,0.0,1.0,36.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Executing,1.0,6.0,1.0,0.0,40.0,1.0,2.0
1,Released,10.0,32.0,0.0,1.0,36.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Executing,1.0,6.0,1.0,0.0,40.0,1.0,2.0
1,Released,10.0,32.0,0.0,1.0,36.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()